In [2]:
import pandas as pd
from rapidfuzz import process, utils, fuzz
import re

In [3]:
new_exported_contacts_df = pd.read_csv('/Users/michaelbarski/Desktop/personal_projects/contact-company-matching-poc/making_new_data/new_exported_contacts.csv')
company_data_df = pd.read_csv('/Users/michaelbarski/Desktop/personal_projects/contact-company-matching-poc/making_new_data/company_data.csv')

note -- the company_data_df has added subdomains and typos

In [4]:
contacts_urls = new_exported_contacts_df.company_url
company_urls = company_data_df.company_url

below I'll do the following:
- write the regex pattern
- select the pattern in the company_urls that contains only the base burl -- in this case match.group(2) is the url without a subdomain
- replace the original value for company_url with match.group(2) to get rid of all of the subdomains
- last, use our len(matches) to double check to make sure that we matched all of the records in the data frame (500)

In [5]:
pattern = re.compile(r'(\w+\.)?(\w+\.com)')

matches = []
for val in company_urls:
    match = pattern.fullmatch(val)
    no_subdomain = match.group(2)
    matches.append(no_subdomain)
    company_data_df.company_url.replace({val:no_subdomain}, inplace=True)

print(len(matches))


500


Below I'll merge the two dataframes using an inner join, matching on the company_url column that we just reformatted.

In [6]:
url_matched_df = pd.merge(new_exported_contacts_df, company_data_df, how='inner', on='company_url')



Was able to match 470/500 contacts to companies with the first merge by using regex (94%).
We don't have a 100% match because we intentionally created typos for a number of urls (in an effort to simulate a messier dataset).

There are many ways to match and merge the rest of the data, but here I'll showcase some approximate (or 'fuzzy') matching on company_name. This can be useful if there are no company_urls to match on--or, it can be useful if the urls on each seperate dataset are completely different.
- (e.g. holidayinncommunityvillage.com vs hicv.com would be very challenging to match)

Below, I'll create a seperate list for any unmatched records. In the end I'll join these two seperate dataframes.

In [12]:
matched_urls = url_matched_df.company_url
unmatched_contacts_df = new_exported_contacts_df[~new_exported_contacts_df['company_url'].isin(matched_urls)].copy()
unmatched_names_contacts = unmatched_contacts_df.company_name.unique()

cd_company_names = list(company_data_df.company_name)

In [13]:

extracted_matches = []
for name in unmatched_names_contacts:
    fuzzy_match = process.extractOne(query=name, choices=cd_company_names, scorer=fuzz.partial_ratio, score_cutoff=100)
    extracted_matches.append(fuzzy_match)
    print(f'{name} = {fuzzy_match[0]}')


chanay, jeffrey a = chanay, jeffrey a esq
morlong associates = morlong associates
fbs business finance = fbs business finance
vicon corporation = vicon corporation
alpenlite = alpenlite inc
beutelschies & company = beutelschies & company


After looking at the output, we can see that everything worked In this scenario, only 2 of the names were different and all of the matches were correct ones. I've essentailly collected all of the approximate and exact matches in one go. With larger data unclean data sets this situation becomes more involved, but with a simple situation like this, fuzzy matching provides an elegant solution. 

Next, I could replace each name of the two names without a loop, but using the loop allows this process to be more automated when the list content changes, saving a little time.

In [15]:
for i in (range(len(unmatched_names_contacts))):
        to_replace = unmatched_names_contacts[i]
        replace_with = extracted_matches[i][0]
        print(to_replace,replace_with)
        unmatched_contacts_df.company_name.replace(to_replace, replace_with, inplace=True)




chanay, jeffrey a chanay, jeffrey a esq
morlong associates morlong associates
fbs business finance fbs business finance
vicon corporation vicon corporation
alpenlite alpenlite inc
beutelschies & company beutelschies & company


Then we create another df for the names we matched and reformat the columns so we can combine the two merged dfs.

In [16]:
name_matched_df = pd.merge(unmatched_contacts_df,company_data_df, 'inner', on='company_name').drop('company_url_y', axis=1)
name_matched_df.rename(columns={'company_url_x':'company_url'}, inplace=True)
url_matched_df.drop(['company_name_y'], axis = 1, inplace=True)
url_matched_df.rename(columns={'company_name_x':'company_name'}, inplace=True)

Last we stack both of these dataframes on top of each other to create a complete fully_matched list of company/contact records. Then I rearrange the columns to make it look more uniform.

In [21]:
fully_matched_df = pd.concat([url_matched_df, name_matched_df], axis=0)

column_to_move = fully_matched_df.pop('company_name')
fully_matched_df.insert(8, 'company_name', column_to_move)

In [18]:
fully_matched_df.to_csv('finished_contact_records.csv')

We matched all of the records and merged them successfully! We've associated company data with every contact record. Now we can upload these new records to salesforce--or whichever crm we'd like--and our salespeople can see the company information (like revenue and what tech the company uses) for every contact; now it's all available in one place!